## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
import re

In [ ]:
rt_movie_info = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep = '\t')

rt_movie_info.dropna(subset = ['box_office'], inplace = True)

for col in ['box_office']:
    rt_movie_info[col] = rt_movie_info[col].str.replace(r'\D', '')
    rt_movie_info[col] = rt_movie_info[col].astype('int64')
    
rt_10pct_box_office = rt_movie_info['box_office'].quantile(.9)

top_27_directors = rt_movie_info.groupby(['director'])['box_office'].mean().sort_values(ascending = False).head(27)

#visual
sns.barplot(top_27_directors.index, top_27_directors.values)

In [ ]:
kaggle_movies = pd.read_csv('./kaggleData/movie_production.csv', encoding = 'latin1')

kaggle_movies['profit'] = kaggle_movies['gross'] - kaggle_movies['budget']

kag_company_vs_prof = kaggle_movies.groupby(['company'])['profit'].mean().sort_values(ascending = False).head(20)

#visual
plt.figure(figsize = (20,6))
plt.title('Top 20 Most Profitable Movie Studios')
studios_plot = sns.barplot(x = kag_company_vs_prof.index, y = kag_company_vs_prof.values, palette = 'Blues_r')
studios_plot.set_xticklabels(studios_plot.get_xticklabels(), rotation = 70);
#visual


# Cleaning Ratings Column

kaggle_movies['rating'] = kaggle_movies['rating'].map(lambda x: 'NOT RATED' if x == 'Not specified' else x)
kaggle_movies['rating'] = kaggle_movies['rating'].map(lambda x: 'R' if x == 'NC-17' else x)

for rating in ['B15', 'TV-MA', 'TV-PG', 'TV-14', 'B']:
    kaggle_movies.drop(kaggle_movies[kaggle_movies['rating'] == rating].index, inplace = True)
    
#

#visual
rating_count_barplot = sns.barplot(kaggle_movies['rating'].value_counts().index, 
                                   kaggle_movies['rating'].value_counts().values, 
                                   palette = 'mako')


rating_vs_mean_profit_barplot = sns.barplot(kaggle_movies['rating'], 
                                            kaggle_movies['profit'],  
                                            palette = 'mako', 
                                            order = ['R', 'PG-13', 'PG', 'NOT RATED','G','UNRATED'])

#


#visual
def genre_vs_profit_barplot(dataframe, genre, profit):
    plt.title('Movie Genre vs Profit')
    chart = sns.barplot(genre, 
                        profit, 
                        data = dataframe)
    chart.set_xticklabels(chart.get_xticklabels(), rotation = 60)
    chart.set(xlabel='Movie Genre', ylabel='Profit ($10M)');
    return plt.show()


def create_kaggle_ratings_dfs_for_subplots(dataframe):
    """
    
    Takes kaggle_movies and returns six dfs, one for each rating.
    
    """
    ratings_list = ['R', 'PG-13', 'PG', 'NOT RATED','G','UNRATED']
    dfs_list = []
    
    for rating in range(len(ratings_list)):
        df = dataframe[dataframe['rating'] == ratings_list[rating]]
        dfs_list.append(df)
    
    return dfs_list

    

# def list_to_six_subplot_tuple(dfs_list):
    
#     """
#     Converts list of dataframes returned by create_kaggle_ratings_dfs_for_subplots()
#     into a tuple formatted specifically for the subplots function called
#     create_ratings_subplots_from_dfs_list().
#     """
    
#     nested_df_list = [[], []]
#     for j in range(len(nested_df_list)):
#         for i in range(len(dfs_list)):
#             if i < (len(dfs_list)/2):
#                 nested_df_list[j].append(dfs_list[i])
#             else:
#                 nested_df_list[j].append(dfs_list[i])
#     return nested_df_list
 
    

   
    
# def create_ratings_subplots_from_dfs_list(dfs_list):
    
#     """
#     Takes list of dataframes, one for each rating, and creates a subplot for each.
#     """
    
#     axes_tuple = ((ax1, ax2, ax3), (ax4, ax5, ax6))
#     plot_list = []
#     fig, axes_tuple = plt.subplots(ncols=3, nrows = 2, sharey = True, sharex = True); 
    
#     for a in range(len(axes_tuple)):
#         for axes in range(len(axes_tuple[a])):
#             plot = sns.regplot(dfs_list[a][axes]['runtime'], dfs_list[a][axes]['profit'], ax = axes_tuple[a][axes])
#             plot_list.append(plot)
    
#     return plot_list

    
def ratings_subplots(dfs_list):
    """
    Hardcoded subplots for each of the six MPAA movie ratings.
    """

    
    
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(ncols=3, 
                                              nrows = 2,  
                                              figsize = (18,10))
    
    axes_tuple = ((ax1, ax2, ax3), (ax4, ax5, ax6))
    
    fig.suptitle('Profit vs. Runtime for each MPAA Rating')
#     for a in range(len(axes_tuple)):
#         for axes in range(len(axes_tuple[a])):
#             axes_tuple[a][axes].set_xlabel('Runtime (Minutes)')
#             axes_tuple[a][axes].set_ylabel('Profit ($10M)')
            
    fig.suptitle('Profit vs. Runtime for each MPAA Rating')

    sns.regplot(x=dfs_list[0]['runtime'], y=dfs_list[0]['profit'], ax=ax1, color = 'black')
    sns.regplot(x=dfs_list[1]['runtime'], y=dfs_list[1]['profit'], ax=ax2, color = 'darkslateblue')
    sns.regplot(x=dfs_list[2]['runtime'], y=dfs_list[2]['profit'], ax=ax3, color = 'steelblue')
    sns.regplot(x=dfs_list[3]['runtime'], y=dfs_list[3]['profit'], ax=ax4, color = 'teal')
    sns.regplot(x=dfs_list[4]['runtime'], y=dfs_list[4]['profit'], ax=ax5, color = 'c')
    sns.regplot(x=dfs_list[5]['runtime'], y=dfs_list[5]['profit'], ax=ax6, color = 'palegreen')

    for a in range(len(axes_tuple)):
        for axes in range(len(axes_tuple[a])):
            axes_tuple[a][axes].set_xlabel('Runtime (Minutes)')
            axes_tuple[a][axes].set_ylabel('Profit ($10M)')
    
    ax1.set_title('R')
    ax2.set_title('PG-13')
    ax3.set_title('PG')
    ax4.set_title('NOT RATED')
    ax5.set_title('G')
    ax6.set_title('UNRATED')
    
